# Importing libraries

In [55]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

# Importing and preprocessing data

In [56]:
mnist_dataset, mnist_info = tfds.load(name = 'mnist', with_info = True, as_supervised = True)

In [57]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']
num_validation_samples = 0.1* mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, np.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, np.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image/=255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)

test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

# Model

In [63]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape = (28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(output_size, activation = 'softmax'),
])

# Optimizer and loss function

In [64]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics = ['accuracy'])

# Training

In [65]:
NUM_EPOCHS = 5

model.fit(train_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), validation_steps=1,verbose=2)

Epoch 1/5
540/540 - 10s - loss: 0.3367 - accuracy: 0.9044 - val_loss: 0.1912 - val_accuracy: 0.9470
Epoch 2/5
540/540 - 10s - loss: 0.1424 - accuracy: 0.9584 - val_loss: 0.1262 - val_accuracy: 0.9643
Epoch 3/5
540/540 - 9s - loss: 0.0994 - accuracy: 0.9700 - val_loss: 0.1034 - val_accuracy: 0.9707
Epoch 4/5
540/540 - 10s - loss: 0.0752 - accuracy: 0.9775 - val_loss: 0.0891 - val_accuracy: 0.9733
Epoch 5/5
540/540 - 8s - loss: 0.0615 - accuracy: 0.9813 - val_loss: 0.0697 - val_accuracy: 0.9815


# Testing on test dataset

In [66]:
test_loss, test_accuracy = model.evaluate(test_data)

      1/Unknown - 2s 2s/step - loss: 0.0900 - accuracy: 0.9720

In [67]:
print('Test loss: {0:.2f}. Test Accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100))

Test loss: 0.09. Test Accuracy: 97.20%
